<a href="https://colab.research.google.com/github/adilsonalbino/SQL-e-PYSPARK/blob/main/15_Fun%C3%A7%C3%A3o_ROW_NUMBER_com_SQL_e_PYSPARK.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#15-Função ROWNUMBER com SQL e PYSPARK

by Adilson Albino

In [4]:
#instalando pyspark
!pip install pyspark

#importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

#Criando app spark
spark = SparkSession.builder \
.appName("Spark")\
.getOrCreate()

In [3]:
#Variavel com a informação do caminho do dataset
file_path_carros = "/content/drive/MyDrive/Colab Notebooks/PySpark_SQL/DATASET_CARROS/modelo_carro.csv"

In [6]:
#Carregando o dataset
df_carros = spark\
.read.format("csv")\
.option("header", True)\
.load(file_path_carros)\

df_carros = df_carros.withColumn(
    "preco",
    regexp_replace("preco", "\$", "")
    .cast(DoubleType())
)


df_carros.show(10)

+--------+------------+--------+---------+
|id_carro|modelo_carro|   preco|cod_marca|
+--------+------------+--------+---------+
|       1|      Avalon|78401.95|       54|
|       2|         RDX|95987.38|        1|
|       3|        Golf|61274.55|       55|
|       4|          EX|84981.12|       23|
|       5|      Escort|77466.89|       17|
|       6|  Expedition|84698.71|       17|
|       7|     Voyager|95567.75|       42|
|       8|       Civic|84749.22|       20|
|       9|    Defender|98600.79|       29|
|      10|V8 Vantage S|94791.61|        2|
+--------+------------+--------+---------+
only showing top 10 rows



#Utilizando SQL

In [7]:
#Criando tabela temporária
df_carros.createOrReplaceTempView("carros")

In [9]:
df_carros_sql = spark.sql("""
    SELECT
      *,
      ROW_NUMBER() OVER (PARTITION BY modelo_carro ORDER BY preco DESC) AS RowNumber
    FROM carros

""")

df_carros_sql.show(10)

+--------+---------------+--------+---------+---------+
|id_carro|   modelo_carro|   preco|cod_marca|RowNumber|
+--------+---------------+--------+---------+---------+
|     117|            100|55023.26|        3|        1|
|      71|           1000|82354.14|       43|        1|
|      57|           1500|70293.96|       10|        1|
|     634|           1500|45539.22|       18|        2|
|     882|1500 Club Coupe|73878.68|       18|        1|
|     690|          200SX|76179.86|       40|        1|
|     171|            240|52885.44|       56|        1|
|     283|          240SX|94383.87|       40|        1|
|     136|          240SX|87853.77|       40|        2|
|     891|          240SX|74588.41|       40|        3|
+--------+---------------+--------+---------+---------+
only showing top 10 rows



#Utilizando pyspark


In [11]:
#importando o windows
from pyspark.sql.window import Window

In [13]:

df_carros_spark = df_carros.withColumn(
    "RowNumber",
    row_number()
    .over(Window
    .partitionBy("modelo_carro")
    .orderBy(desc("preco"))
    )
)

df_carros_spark.show(10)

+--------+---------------+--------+---------+---------+
|id_carro|   modelo_carro|   preco|cod_marca|RowNumber|
+--------+---------------+--------+---------+---------+
|     117|            100|55023.26|        3|        1|
|      71|           1000|82354.14|       43|        1|
|      57|           1500|70293.96|       10|        1|
|     634|           1500|45539.22|       18|        2|
|     882|1500 Club Coupe|73878.68|       18|        1|
|     690|          200SX|76179.86|       40|        1|
|     171|            240|52885.44|       56|        1|
|     283|          240SX|94383.87|       40|        1|
|     136|          240SX|87853.77|       40|        2|
|     891|          240SX|74588.41|       40|        3|
+--------+---------------+--------+---------+---------+
only showing top 10 rows

